In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

Kernel density estimation

https://en.wikipedia.org/wiki/Kernel_density_estimation  
https://sebastianraschka.com/Articles/2014_kernel_density_est.html  

## Байесовский бутстрап

https://projecteuclid.org/journals/annals-of-statistics/volume-9/issue-1/The-Bayesian-Bootstrap/10.1214/aos/1176345338.full  
https://stats.stackexchange.com/questions/181350/bootstrapping-vs-bayesian-bootstrapping-conceptually  
https://matteocourthoud.github.io/post/bayes_boot/  

Есть выборка.  
Считается частота точек.  
Получается 

$$
P(n_1, ..., n_k) = Mult(n_1, ..., n_k | n_1/N, ..., n_k/N)  
$$

Мультиномиальное распределение меняется на распределение Дирихле

$$
Dir()
$$

In [ ]:
nsample = 15000

a = 2
b = 3
exact_dist = stats.gamma(a=a, scale=1/b)
data = exact_dist.rvs(nsample)

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
fig.update_layout(title='Точное',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

In [ ]:
unique, counts = np.unique(x, return_counts=True)

print(np.asarray((unique, counts)).T)

## Дискретизация произвольных распределений

1) Интервалы  
Выбираются дискретные интервалы.  
Правдоподобие - мультиномиальное распределение.  

Равномерное распределение внутри интервала?  

Подбирать ширину интервалов?
По 1 тыс. точек.

Еще можно последовательно разбивать интервал на меньшие.  
Как в решающих деревьях.

Правдоподобие:
для каждой точки выбрать интервал, выбрать точку внутри интервала.   
Вероятность i-го интервала - категориальное распределение https://en.wikipedia.org/wiki/Categorical_distribution .  
Домножить либо на среднее. Либо на точку из интервала.  

$$
P(\mathcal{D} | \mathcal{H}) = \prod_N \mbox{Uniform}(l_i, r_i) \mbox{Categorial}(i | p_1, \dots, p_k)
$$

Другой вариант - количество точек в i-ом интервале на среднее интервала

$$
P(\mathcal{D} | \mathcal{H}) = (n_1 \mu_1 + \dots + n_k \mu_k) \mbox{Mult}(n_1, \dots, n_k | p_1, \dots, p_k)
$$



In [ ]:
nsample = 15000

a = 2
b = 3
exact_dist = stats.gamma(a=a, scale=1/b)
data = exact_dist.rvs(nsample)

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
fig.update_layout(title='Точное',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

In [ ]:
bins = 30
hd = np.histogram(data, bins=bins, density=True)
c = hd[0]
b = [(l, r) for l, r in zip(hd[1][:-1], hd[1][1:])]
#b, c


#print(b)

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
fig.add_trace(go.Bar(x=[(w[0]+w[1])/2 for w in b], y=c, 
                      name='Hist'))
fig.update_layout(title='Точное',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

In [ ]:
hd

In [ ]:
stats.binned_statistic(data, data, statistic='mean', bins=bins)

# bins = np.linspace(0, 1, 10)
# digitized = np.digitize(data, bins)
# bin_means = [data[digitized == i].mean() for i in range(1, len(bins))]
# bin_means

# bin_means = (np.histogram(data, bins=bins, weights=data)[0] / np.histogram(data, bins=bins)[0])
# bin_means

Аналогично заказам на пользователя

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = Mult(n_0, \dots, n_N | p_0, \dots, p_N) = \frac{(n_0 + \dots + n_N)!}{n_{0}! \dots n_{N}!} p_{0}^{n_{0}} \dots p_{N}^{n_{N}} 
\\
P(\mathcal{H}) & = 
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right) = 
\dfrac{1}{B( \alpha_{0}, \dots, \alpha_{N} )} \prod_{i=0}^{N} p_{i}^{\alpha_{i}-1},
\qquad
\sum_{i=0}^{N} p_i = 1,
\qquad
p_i \in [0, 1], 
\qquad
B(\alpha_{0}, \dots, \alpha_{N}) = 
\frac{\prod \limits_{i=0}^{N} \Gamma( \alpha_{i} )}
{\Gamma \left( \sum \limits_{i=0}^{N} \alpha_{i} \right)}
\\
P(\mathcal{H} | \mathcal{D}) 
& \propto Mult(n_0, \dots, n_N | p_0, \dots, p_N) Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right)
\\
& \propto
p_{0}^{n_{0}} \dots p_{N}^{n_{N}} 
\prod _{i=0}^{N} p_{i}^{\alpha_{i}-1}
\\
& \propto
\prod_{i=0}^{N} p_{i}^{n_{i} + \alpha_{i} - 1}
\\
& =
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0} + n_0, \dots, \alpha_{N} + n_N \right)
\\
P(p_i | \mathcal{D} ) & = 
\int dp_0 \dots dp_{i-1}dp_{i+i} \dots dp_N P(\mathcal{H} | \mathcal{D}) 
=
Beta( p_i; \alpha_i + n_i, \sum_{k=0}^{N} (\alpha_k + n_k) - \alpha_i - n_i )
\end{split}
$$

In [ ]:
def initial_params_dir(N):
    return np.ones(N)

def posterior_params_dir(bin_counts, initial_pars):
    post_pars = np.copy(initial_pars)
    post_pars = post_pars + bin_counts
    return post_pars

def posterior_dist_dir(params):
    return stats.dirichlet(alpha=params)

def posterior_bins_dir_rvs(params, nsamp):
    bins = np.empty(nsamp)
    d = posterior_dist_dir(params)
    probs = d.rvs(size=nsamp)
    for i, p in enumerate(probs):
        bins[i] = np.argmax(stats.multinomial.rvs(n=1, p=p))
    return bins

def posterior_bin_unif_dir_rvs(params, bins_lr, nsamp):
    post = np.empty(nsamp)
    d = posterior_dist_dir(params)
    probs = d.rvs(size=nsamp)
    for i, p in enumerate(probs):
        b_i = np.argmax(stats.multinomial.rvs(n=1, p=p))
        l, r = bins_lr[b_i]
        post[i] = stats.uniform.rvs(loc=l, scale=r-l)
    return post

def posterior_bins_mean_rvs(params, bins_mean, nsample):
    probs = stats.dirichlet.rvs(alpha=params, size=nsample)
    means = np.sum(bins_mean * probs, axis=1)
    return means

def marginal_pi_dist_dir(i, params):
    return stats.beta(a=params[i], b=np.sum(params) - params[i])

def posterior_pi_mean_95pdi(i, params):
    p = marginal_pi_dist_dir(i, params)
    m = p.mean()
    lower = p.ppf(0.025)
    upper = p.ppf(0.975)
    return m, lower, upper

nsample = 15000

a = 2
b = 3
exact_dist = stats.gamma(a=a, scale=1/b)
data = exact_dist.rvs(nsample)
n_bins = 30
bins = np.histogram(data, bins=n_bins, density=False)
dn = np.histogram(data, bins=n_bins, density=True)[0]
cnt = bins[0]
b_width = [(l, r) for l, r in zip(bins[1][:-1], bins[1][1:])]

#display(cnt)

pars = initial_params_dir(n_bins)
pars = posterior_params_dir(cnt, pars)
#pars
post_samp = posterior_nords_dir_rvs(pars, 100000)
pi = [posterior_pi_mean_95pdi(i, pars) for i in range(n_bins)]

x = np.linspace(0, 3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
fig.add_trace(go.Bar(x=[(w[0] + w[1])/2 for w in b_width], y=dn,
                     opacity=0.4,
                      name='Hist'))
# fig.add_trace(go.Histogram(x=post_samp, histnorm='probability', name='$\mbox{Апостериорные } n_i$', 
#                          marker_color='black', opacity=0.2, nbinsx=round(Nmax*2)))
fig.add_trace(go.Scatter(x=[(w[0] + w[1])/2 for w in b_width], 
                         y=np.array([p[0] for p in pi]) / np.array([w[1] - w[0] for w in b_width]),
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi] / np.array([w[1] - w[0] for w in b_width]), arrayminus=[p[0] - p[1] for p in pi] / np.array([w[1] - w[0] for w in b_width])), 
                         name='$\mbox{Оценки } p_i$',
                         mode='markers',
                         line_color='red',
                         opacity=0.8))
fig.update_layout(#title='Заказы на посетителя',
                  #xaxis_title='$Заказы$',
                  #yaxis_title='Вероятность',
                  #xaxis_range=[-1, Nmax+1],
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()

Апостериорное среднее - сгенерировать интервал, потом равномерно внутри интервала.

Можно записать аналитически: сумма по интервалам (среднее в интервале * вероятность).

$$
E[x] = \sum_i E_i p_i
$$

In [ ]:
npostsamp = 10000
post_samp = posterior_bin_unif_dir_rvs(pars, b_width, npostsamp)

#scipy bin statistics
nmeanssamp = 1000
b_means = [(x[0] + x[1])/2 for x in b_width]
post_means = posterior_bins_mean_rvs(pars, b_means, nmeanssamp)

x = np.linspace(0, 3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
# fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Выборка', nbinsx=500,
#                            marker_color='black', opacity=0.3))
# fig.add_trace(go.Bar(x=[(w[0] + w[1])/2 for w in b_width], y=dn,
#                      opacity=0.4,
#                       name='Hist'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='Post samp', 
                         marker_color='black', opacity=0.2, nbinsx=300))
# fig.add_trace(go.Scatter(x=[(w[0] + w[1])/2 for w in b_width], 
#                          y=np.array([p[0] for p in pi]) / np.array([w[1] - w[0] for w in b_width]),
#                          error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi] / np.array([w[1] - w[0] for w in b_width]), arrayminus=[p[0] - p[1] for p in pi] / np.array([w[1] - w[0] for w in b_width])), 
#                          name='$\mbox{Оценки } p_i$',
#                          mode='markers',
#                          line_color='red',
#                          opacity=0.8))
fig.update_layout(#title='Заказы на посетителя',
                  #xaxis_title='$Заказы$',
                  #yaxis_title='Вероятность',
                  #xaxis_range=[-1, Nmax+1],
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()


x = np.linspace(0, 2, 1000)
fig = go.Figure()
#fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
#                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=post_means, histnorm='probability', name='Post means', 
                         marker_color='green', opacity=0.2, nbinsx=30))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], 
                         y=[0, np.max(exact_dist.pdf(x))*1.1],
                         name='Exact mean', 
                         mode='lines', line_dash='dash',
                         line_color='black'))
fig.update_layout(#title='Заказы на посетителя',
                  #xaxis_title='$Заказы$',
                  #yaxis_title='Вероятность',
                  #xaxis_range=[-1, Nmax+1],
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()

In [ ]:
def posterior_bins_mean_rvs(params, bins_mean, nsample):
    probs = stats.dirichlet.rvs(alpha=params, size=nsample)
    means = np.sum(bins_mean * probs, axis=1)
    return means

# def prob_pb_gt_pa_samples(post_samp_A, post_samp_B):
#     if len(post_samp_A) != len(post_samp_B):
#         return None
#     b_gt_a = np.sum(post_samp_B > post_samp_A)
#     return b_gt_a / len(post_samp_A)

# nsample = 3000
# Nmax = 30
# Npars = Nmax + 1

# post_samp_len = 100000
# A, B = {}, {}
# s = 1.5
# A['dist_pars'] = {'s': s}
# B['dist_pars'] = {'s': s * 0.95}
# for g in [A, B]:
#     g['exact_dist'] = stats.zipfian(a=g['dist_pars']['s'], n=Npars, loc=-1)
#     g['data'] = g['exact_dist'].rvs(nsample)
#     g['post_pars'] = initial_params_dir(Npars)
#     g['post_pars'] = posterior_params_dir(g['data'], g['post_pars'])
#     g['post_nords'] = posterior_nords_dir_rvs(g['post_pars'], post_samp_len)
#     g['post_means'] = posterior_nords_mean_rvs(g['post_pars'], post_samp_len)
#     g['pi'] = [posterior_pi_mean_95pdi(i, g['post_pars']) for i in range(Npars)]

# x = np.arange(0, Npars)
# fig = go.Figure()
# fig.add_trace(go.Bar(x=x, y=A['exact_dist'].pmf(x), name='Точное распределение A',
#                         marker_color='black', opacity=0.2))
# fig.add_trace(go.Bar(x=x, y=B['exact_dist'].pmf(x), name='Точное распределение Б',
#                         marker_color='black', opacity=0.8))
# fig.add_trace(go.Scatter(x=[A['exact_dist'].mean(), A['exact_dist'].mean()], 
#                          y=[0, np.max(A['exact_dist'].pmf(x))*1.1],
#                          name='Точное среднее A', 
#                          mode='lines', line_dash='dash',
#                          line_color='black', opacity=0.3))
# fig.add_trace(go.Scatter(x=[B['exact_dist'].mean(), B['exact_dist'].mean()], 
#                          y=[0, np.max(B['exact_dist'].pmf(x))*1.1],
#                          name='Точное среднее Б', 
#                          mode='lines', line_dash='dash',
#                          line_color='black'))
# fig.add_trace(go.Scatter(x=x - 0.1, 
#                          y=[p[0] for p in A['pi']],
#                          error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in A['pi']], arrayminus=[p[0] - p[1] for p in A['pi']]), 
#                          name='$p_i, \mbox{ А}$',
#                          line_color='black', opacity=0.3,
#                          mode='markers'
#                     ))
# fig.add_trace(go.Scatter(x=x + 0.1, 
#                          y=[p[0] for p in B['pi']],
#                          error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in B['pi']], arrayminus=[p[0] - p[1] for p in B['pi']]), 
#                          name='$p_i, \mbox{ Б}$',
#                          line_color='black',
#                          mode='markers'))
# fig.update_layout(title='Заказы на посетителя',
#                   xaxis_title='$Заказы$',
#                   yaxis_title='Вероятность',
#                   xaxis_range=[-1, Npars+1-20],
#                   hovermode="x",
#                   barmode="group",
#                   height=550)
# fig.show()
# #fig.write_image("./figs/ch6_cmp_orig.png", scale=2)
# #Точные распределения, точные средние количества заказов и оценки конверсий.

# x = np.arange(0, Npars)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=[A['exact_dist'].mean(), A['exact_dist'].mean()], 
#                          y=[0, np.max(A['exact_dist'].pmf(x))*1.1],
#                          name='Точное среднее A', 
#                          mode='lines', line_dash='dash',
#                          line_color='black', opacity=0.3))
# fig.add_trace(go.Scatter(x=[B['exact_dist'].mean(), B['exact_dist'].mean()], 
#                          y=[0, np.max(B['exact_dist'].pmf(x))*1.1],
#                          name='Точное среднее Б', 
#                          mode='lines', line_dash='dash',
#                          line_color='black'))
# fig.add_trace(go.Histogram(x=A['post_means'], histnorm='probability', name='$E[n], \mbox{ А}$', 
#                            marker_color='black', opacity=0.3, nbinsx=round(Nmax*2)))
# fig.add_trace(go.Histogram(x=B['post_means'], histnorm='probability', name='$E[n], \mbox{ Б}$', 
#                            marker_color='black', nbinsx=round(Nmax*2)))
# fig.update_layout(title='Среднее количество заказов',
#                   xaxis_title='$Заказы$',
#                   yaxis_title='Вероятность',
#                   xaxis_range=[-1, Npars+1-20],
#                   hovermode="x",
#                   barmode="group",
#                   height=550)
# fig.show()
# #fig.write_image("./figs/ch6_cmp_means.png", scale=2)
# #Оценки среднего количества заказов. Среднее Б выше А с вероятностью 90%.

# print(f"P(E[n]_B > E[n]_A): {prob_pb_gt_pa_samples(A['post_means'], B['post_means'])}")

Если уменьшать интервалы до отдельных точек будет аналог бутстрапа.  
Интервалы учитывают предположение непрерывности.  
Можно ли построить то же по точкам?
Точек много. Тоже группировать?

Вместо плотности вероятности может быть удобнее смотреть функцию распределения.

Бины с фиксированным количеством точек. Например, 1000.  
Но не шире максимальной ширины (могут быть проблемы с хвостом).

2) Сумма нормальных?  
https://en.wikipedia.org/wiki/Radial_basis_function  
https://en.wikipedia.org/wiki/Radial_basis_function_interpolation

In [ ]:
def posterior_nords_mean_rvs(params, nsample):
    ns = np.arange(len(params))
    probs = stats.dirichlet.rvs(alpha=params, size=nsample)
    means = np.sum(ns * probs, axis=1)
    return means

def prob_pb_gt_pa_samples(post_samp_A, post_samp_B):
    if len(post_samp_A) != len(post_samp_B):
        return None
    b_gt_a = np.sum(post_samp_B > post_samp_A)
    return b_gt_a / len(post_samp_A)

nsample = 3000
Nmax = 30
Npars = Nmax + 1

post_samp_len = 100000
A, B = {}, {}
s = 1.5
A['dist_pars'] = {'s': s}
B['dist_pars'] = {'s': s * 0.95}
for g in [A, B]:
    g['exact_dist'] = stats.zipfian(a=g['dist_pars']['s'], n=Npars, loc=-1)
    g['data'] = g['exact_dist'].rvs(nsample)
    g['post_pars'] = initial_params_dir(Npars)
    g['post_pars'] = posterior_params_dir(g['data'], g['post_pars'])
    g['post_nords'] = posterior_nords_dir_rvs(g['post_pars'], post_samp_len)
    g['post_means'] = posterior_nords_mean_rvs(g['post_pars'], post_samp_len)
    g['pi'] = [posterior_pi_mean_95pdi(i, g['post_pars']) for i in range(Npars)]

x = np.arange(0, Npars)
fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=A['exact_dist'].pmf(x), name='Точное распределение A',
                        marker_color='black', opacity=0.2))
fig.add_trace(go.Bar(x=x, y=B['exact_dist'].pmf(x), name='Точное распределение Б',
                        marker_color='black', opacity=0.8))
fig.add_trace(go.Scatter(x=[A['exact_dist'].mean(), A['exact_dist'].mean()], 
                         y=[0, np.max(A['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее A', 
                         mode='lines', line_dash='dash',
                         line_color='black', opacity=0.3))
fig.add_trace(go.Scatter(x=[B['exact_dist'].mean(), B['exact_dist'].mean()], 
                         y=[0, np.max(B['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее Б', 
                         mode='lines', line_dash='dash',
                         line_color='black'))
fig.add_trace(go.Scatter(x=x - 0.1, 
                         y=[p[0] for p in A['pi']],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in A['pi']], arrayminus=[p[0] - p[1] for p in A['pi']]), 
                         name='$p_i, \mbox{ А}$',
                         line_color='black', opacity=0.3,
                         mode='markers'
                    ))
fig.add_trace(go.Scatter(x=x + 0.1, 
                         y=[p[0] for p in B['pi']],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in B['pi']], arrayminus=[p[0] - p[1] for p in B['pi']]), 
                         name='$p_i, \mbox{ Б}$',
                         line_color='black',
                         mode='markers'))
fig.update_layout(title='Заказы на посетителя',
                  xaxis_title='$Заказы$',
                  yaxis_title='Вероятность',
                  xaxis_range=[-1, Npars+1-20],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()
#fig.write_image("./figs/ch6_cmp_orig.png", scale=2)
#Точные распределения, точные средние количества заказов и оценки конверсий.

x = np.arange(0, Npars)
fig = go.Figure()
fig.add_trace(go.Scatter(x=[A['exact_dist'].mean(), A['exact_dist'].mean()], 
                         y=[0, np.max(A['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее A', 
                         mode='lines', line_dash='dash',
                         line_color='black', opacity=0.3))
fig.add_trace(go.Scatter(x=[B['exact_dist'].mean(), B['exact_dist'].mean()], 
                         y=[0, np.max(B['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее Б', 
                         mode='lines', line_dash='dash',
                         line_color='black'))
fig.add_trace(go.Histogram(x=A['post_means'], histnorm='probability', name='$E[n], \mbox{ А}$', 
                           marker_color='black', opacity=0.3, nbinsx=round(Nmax*2)))
fig.add_trace(go.Histogram(x=B['post_means'], histnorm='probability', name='$E[n], \mbox{ Б}$', 
                           marker_color='black', nbinsx=round(Nmax*2)))
fig.update_layout(title='Среднее количество заказов',
                  xaxis_title='$Заказы$',
                  yaxis_title='Вероятность',
                  xaxis_range=[-1, Npars+1-20],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()
#fig.write_image("./figs/ch6_cmp_means.png", scale=2)
#Оценки среднего количества заказов. Среднее Б выше А с вероятностью 90%.

print(f"P(E[n]_B > E[n]_A): {prob_pb_gt_pa_samples(A['post_means'], B['post_means'])}")